# General Analysis for the eFEDS-XCS Paper

In this notebook we perform all of the analysis required for the eFEDS-XCS comparison paper, including generating many of the figures and tables in the paper. This analysis includes comparing eFEDS distributions to XXL measured distributions, measuring XMM temperatures and luminosities for clusters in the eFEDS-XCS sample, and fitting luminosity-temperature relations (both using XMM and eFEDS data).

The sample which we analyse in this notebook (eFEDS-XCS), only contains candidates that we consider to be galaxy clusters, and that have a high enough quality of XMM observation (more information is supplied in the paper). The sources in the eFEDS-XMM and eFEDS-XCS samples are detailed in the 'efeds_xmm.csv' and 'efeds_xcs.csv' files, and the 'excluded_cands.csv' file contains further information about which eFEDS-XMM candidates were not included in the final sample and why.

We attempt to present the code in the same order that results are presented in the paper. Any enquires about specific parts of the analysis are welcome, please contact me (D.J. Turner) at the email address provided in the paper.

## Import Statements

In [3]:
import pandas as pd
from astropy.units import Quantity
from astropy.cosmology import LambdaCDM
from matplotlib import pyplot as plt
from matplotlib.ticker import FuncFormatter
import numpy as np
import scipy.odr as odr
from tqdm import tqdm
from copy import deepcopy

import xga
from xga.samples.extended import ClusterSample
from xga.sources import GalaxyCluster
from xga.xspec import single_temp_apec
from xga.sas import eexpmap, emosaic
from xga.relations.fit import scaling_relation_lira, _fit_initialise
from xga.sourcetools.match import simple_xmm_match
from xga.models import straight_line, convert_to_odr_compatible, power_law
from xga import CENSUS, XSPEC_VERSION, SAS_VERSION
from xga.exceptions import ModelNotAssociatedError
from xga.products.relation import ScalingRelation

## Confirming software versions

Here we display the software versions used in this analysis:

In [5]:
print("Using XGA v{}".format(xga.__version__))
print("Using SAS v{}".format(SAS_VERSION))
print("Using XSPEC v{}".format(XSPEC_VERSION))

Using XGA v0+unknown
Using SAS v17.0.0
Using XSPEC v12.10.1


## Setting up the cosmology to use

As stated in the introduction of the paper, we use a concordance ΛCDM cosmology where Ω$_\rm{M}$=0.3, Ω$_\rm{Λ}$=0.7, and H$_0$=70 km s$^{−1}$Mpc$^{−1}$, consistent with the original eFEDS cluster analysis (and other XCS works)

In [ ]:
cosmo = LambdaCDM(70, 0.3, 0.7)

## Loading the samples

In this section we load the various samples required for this analysis:

### Reading in the XXL-100-GC catalogue

This is the catalogue of the brightest 100 clusters detected by the XXL survey, and is publically [available on VizieR](https://vizier.u-strasbg.fr/viz-bin/VizieR-3?-source=IX/49/xxl100gc&-out.max=50&-out.form=HTML%20Table&-out.add=_r&-out.add=_RAJ,_DEJ&-sort=_r&-oc.form=sexa). It contains positional and redshift information, as long as XXL measured temperatures and luminosities. We only take clusters with the flag value equal to zero, which means that the cluster was in the original sample of 100 brightest clusters in XXL paper II, table D.

In [3]:
xxl = pd.read_csv("xxl_gc100.csv")
xxl = xxl[xxl['Flag'] == 0]
xxl.head(10)

,XLSSC,RAJ2000,DEJ2000,z,Ngal,C60,F60,e_F60,Ez,r500MT,...,LXXL300kpc,e_LXXL300kpc,Lbol500MT,e_Lbol500MT,M500MT,e_M500MT,Mgas500,e_Mgas500,Flag,Simbad
0,94,30.648,-6.732,0.886,3,199,4.82,0.44,1.62,0.74,...,19.85,1.71,62.01,5.35,NaN,NaN,1.61,0.59,0,Simbad
1,96,30.973,-5.027,0.520,6,161,3.64,0.39,1.31,1.00,...,3.77,0.40,16.05,1.71,48.0,31.0,2.02,0.74,0,Simbad
2,102,31.322,-4.652,0.969,3,199,4.20,0.36,1.69,0.57,...,13.31,1.41,33.56,3.56,19.0,11.0,2.23,0.71,0,Simbad
3,106,31.351,-5.732,0.300,14,681,9.13,0.39,1.16,0.86,...,3.16,0.15,10.48,0.49,24.0,11.0,2.15,0.50,0,Simbad
4,107,31.354,-7.594,0.436,3,263,5.58,0.41,1.25,0.71,...,3.82,0.32,10.33,0.88,15.9,7.6,1.14,0.35,0,Simbad
5,100,31.549,-6.193,0.915,6,124,3.55,0.52,1.64,0.69,...,11.12,2.50,32.42,7.28,26.0,18.0,1.73,0.50,0,Simbad
6,93,31.699,-6.948,0.429,6,418,7.23,0.41,1.24,0.81,...,4.75,0.31,14.91,0.96,23.0,11.0,2.48,0.60,0,Simbad
7,108,31.832,-4.827,0.254,4,451,6.16,0.34,1.13,0.70,...,1.49,0.10,3.86,0.26,12.7,5.6,0.52,0.10,0,Simbad
8,95,31.962,-5.206,0.138,12,141,3.09,0.32,1.06,0.45,...,0.15,0.03,0.24,0.04,2.9,1.3,0.03,0.01,0,Simbad
9,92,32.071,-7.276,0.432,3,166,3.14,0.33,1.24,0.77,...,2.11,0.24,6.24,0.70,20.0,11.0,1.39,0.39,0,Simbad


### Reading in the eFEDS X-ray cluster candidate catalogue

We read in the eFEDS X-ray cluster candidate catalogue that is [available here](https://erosita.mpe.mpg.de/edr/eROSITAObservations/Catalogues/liuA/eFEDS_clusters_V3.fits.gz), though we have converted the file to a csv for convenience. We also alter the resulting table so that it contains $\pm$ uncertainties on $T_{\rm{X}}$ and $L_{\rm{X}}$ measurements, rather than just confidence limits, this makes plotting and fitting easier later on in the analysis. Finally we add a 'name' column based around the 'ID_SRC' values, as the source names in the 'ID' column are not necessarily consistent between the X-ray cluster candidate and optical counterpart catalogues.

In [9]:
efeds_xray = pd.read_csv("efeds_xray_cluster_candidates.csv")

# Changing upper and lower limits to uncertainties for 300kpc values
efeds_xray['T_300kpc-'] = efeds_xray['T_300kpc'] - efeds_xray['T_300kpc_L']
efeds_xray['T_300kpc+'] = efeds_xray['T_300kpc_U'] - efeds_xray['T_300kpc']
efeds_xray['L_300kpc-'] = efeds_xray['L_300kpc'] - efeds_xray['L_300kpc_L']
efeds_xray['L_300kpc+'] = efeds_xray['L_300kpc_U'] - efeds_xray['L_300kpc']
efeds_xray['Lbol_300kpc-'] = efeds_xray['Lbol_300kpc'] - efeds_xray['Lbol_300kpc_L']
efeds_xray['Lbol_300kpc+'] = efeds_xray['Lbol_300kpc_U'] - efeds_xray['Lbol_300kpc']

# Changing upper and lower limits to uncertainties for 500kpc values
efeds_xray['T_500kpc-'] = efeds_xray['T_500kpc'] - efeds_xray['T_500kpc_L']
efeds_xray['T_500kpc+'] = efeds_xray['T_500kpc_U'] - efeds_xray['T_500kpc']
efeds_xray['L_500kpc-'] = efeds_xray['L_500kpc'] - efeds_xray['L_500kpc_L']
efeds_xray['L_500kpc+'] = efeds_xray['L_500kpc_U'] - efeds_xray['L_500kpc']
efeds_xray['Lbol_500kpc-'] = efeds_xray['Lbol_500kpc'] - efeds_xray['Lbol_500kpc_L']
efeds_xray['Lbol_500kpc+'] = efeds_xray['Lbol_500kpc_U'] - efeds_xray['Lbol_500kpc']

efeds_xray['name'] = efeds_xray['ID_SRC'].apply(lambda x: "eFEDS-"+str(x))
efeds_xray.head(10)

,ID,ID_SRC,RA,DEC,EXT_LIKE,DET_LIKE,z,z_type,T_300kpc,T_300kpc_L,...,L_300kpc+,Lbol_300kpc-,Lbol_300kpc+,T_500kpc-,T_500kpc+,L_500kpc-,L_500kpc+,Lbol_500kpc-,Lbol_500kpc+,name
0,eFEDS J082626.6-003429,28993,126.610799,-0.574787,8.486203,5.029723,0.161110,0,-1.000000,-1.000000,...,3.544400e+42,0.000000e+00,1.626600e+43,0.000000,0.000000,0.000000e+00,3.924900e+42,0.000000e+00,1.782000e+43,eFEDS-28993
1,eFEDS J082751.8-002853,11248,126.965471,-0.481638,12.791595,27.865910,0.257160,0,-1.000000,-1.000000,...,1.019100e+43,0.000000e+00,1.721900e+43,0.000000,0.000000,0.000000e+00,1.061100e+43,0.000000e+00,1.796300e+43,eFEDS-11248
2,eFEDS J082808.8-001003,4800,127.036645,-0.167715,28.492811,62.512480,0.076155,0,0.885294,0.786329,...,3.867000e+41,5.808000e+41,5.950000e+41,0.092795,0.093866,5.040000e+41,5.816000e+41,7.621000e+41,9.367000e+41,eFEDS-4800
3,eFEDS J082820.6-000721,4169,127.085556,-0.122752,42.376125,81.378350,0.844900,0,-1.000000,-1.000000,...,3.229000e+43,8.116000e+43,1.281700e+44,0.000000,0.000000,4.183000e+43,4.533000e+43,1.117800e+44,1.806000e+44,eFEDS-4169
4,eFEDS J082840.6-000500,7991,127.169202,-0.083552,18.438711,37.515427,0.319705,0,-1.000000,-1.000000,...,2.502000e+42,6.850000e+42,7.253000e+42,0.000000,0.000000,3.347000e+42,3.113000e+42,8.181000e+42,9.381000e+42,eFEDS-7991
5,eFEDS J082900.0+010756,14973,127.249974,1.132314,19.179321,14.365816,0.354000,0,-1.000000,-1.000000,...,3.956900e+42,1.213300e+43,1.924100e+43,0.000000,0.000000,5.387000e+42,6.003000e+42,2.399600e+43,3.482100e+43,eFEDS-14973
6,eFEDS J082952.7+002140,7528,127.469684,0.361084,7.729656,38.626640,0.420000,0,-1.000000,-1.000000,...,3.770000e+42,1.021200e+43,1.561200e+43,0.000000,0.000000,3.631000e+42,4.122000e+42,1.097500e+43,1.664100e+43,eFEDS-7528
7,eFEDS J082955.5+004132,3810,127.480999,0.692199,24.177258,80.255554,0.939855,0,-1.000000,-1.000000,...,3.137000e+43,1.155100e+44,2.492700e+44,0.000000,0.000000,3.391000e+43,4.018000e+43,1.702800e+44,3.107200e+44,eFEDS-3810
8,eFEDS J083040.7+023220,9837,127.669739,2.538833,11.481833,23.703844,0.111320,0,-1.000000,-1.000000,...,2.945100e+41,3.877500e+41,4.566000e+41,0.000000,0.000000,2.519700e+41,2.993400e+41,3.895400e+41,4.713000e+41,eFEDS-9837
9,eFEDS J083110.6+015616,5601,127.794012,1.937839,11.771409,34.137924,0.419510,0,-1.000000,-1.000000,...,5.381000e+42,1.406700e+43,1.889900e+43,0.000000,0.000000,6.276000e+42,6.880000e+42,1.745500e+43,2.958500e+43,eFEDS-5601


### Reading in the eFEDS cluster candidate optical counterpart catalogue

This catalogue contains information about optical counterparts to eFEDS X-ray selected cluster candidates found by Klein et al. (2021), and is [available here](https://erosita.mpe.mpg.de/edr/eROSITAObservations/Catalogues/klein/eFEDS_c001_main_ctp_clus_v2.1.fits.gz) - again we have converted the file into a csv for convenience. We add an identical 'name' column to that which we added to the X-ray selected cluster candidate catalogue table above.

In [12]:
efeds_opt = pd.read_csv("efeds_cluster_candidate_optical_counterparts.csv")
efeds_opt['name'] = efeds_opt['ID_SRC'].apply(lambda x: "eFEDS-"+str(x))
efeds_opt.head(10)

,Name,ID_SRC,RA_CORR,DEC_CORR,RADEC_ERR_CORR,EXT,EXT_ERR,EXT_LIKE,ML_CTS,ML_CTS_ERR,...,Alpha,Delta,Beta,Ellip,Centershift,DIST_NEXT_OPT_R500,DISTNEXT_XCLUST_MPC,DISTNEXT_XCLUST_R500,MCMF_NWAY_SECTOR,name
0,eFEDS J093712.9+031652,38,144.303568,3.281043,4.000000,26.965470,5.968361,19.696564,83.25920,16.002958,...,0.057188,0.260447,0.291617,0.599467,0.456086,8.338902,7.388609,9.777047,A,eFEDS-38
1,eFEDS J083811.9-015935,53,129.549569,-1.992963,1.381066,8.415080,0.671214,163.666980,840.92730,32.609806,...,0.131410,0.231523,0.211128,0.482397,0.258212,5.109306,6.826096,5.099522,A,eFEDS-53
2,eFEDS J093521.0+023234,82,143.837404,2.542872,2.256955,11.646078,0.921796,175.889470,789.41034,32.895790,...,0.149712,0.214348,0.227254,0.441991,0.313691,99.900002,0.914686,0.694703,A,eFEDS-82
3,eFEDS J092121.2+031726,100,140.338476,3.290604,2.786281,31.717770,1.420627,478.553300,1507.95620,56.981632,...,0.126342,0.216794,0.396560,0.548407,0.253045,99.900002,6.280532,4.680137,A,eFEDS-100
4,eFEDS J085751.7+031039,108,134.465300,3.177539,5.122587,57.368830,1.696070,742.209900,2941.50220,104.567130,...,0.184382,0.298033,0.315817,0.452866,0.265861,1.704293,2.833925,2.126780,A,eFEDS-108
5,eFEDS J092647.5+050033,137,141.697954,5.009053,3.680027,34.240242,1.704842,505.366760,1269.64820,53.113010,...,0.078323,0.137997,0.315292,0.591055,0.260348,99.900002,0.315917,0.230575,A,eFEDS-137
6,eFEDS J084528.7+032739,144,131.369500,3.460887,3.187693,28.416616,1.641801,316.698500,942.22090,43.392887,...,0.130602,0.166659,0.197001,0.046051,0.254389,99.900002,19.738748,15.345604,B1,eFEDS-144
7,eFEDS J092002.2+010220,150,140.009027,1.038868,2.707373,14.811505,1.150945,179.593930,693.43646,32.079433,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,B1,eFEDS-150
8,eFEDS J090131.2+030057,152,135.379989,3.015696,5.006569,33.493958,1.665559,373.949920,1196.78820,51.578068,...,0.158592,0.320781,0.253769,0.489150,0.310878,0.470425,0.486326,0.427307,B1,eFEDS-152
9,eFEDS J083651.3+030002,197,129.213825,3.000636,3.505422,28.246906,3.193915,153.496120,747.97363,42.946740,...,0.150400,0.389033,0.462623,0.140946,0.269811,99.900002,0.165200,0.158356,B1,eFEDS-197


### Reading in the eFEDS-XMM sample

### Reading in the excluded candidate notes

### Reading in the eFEDS-XCS sample

This sample is a subset of the eFEDS-XMM sample, but we felt that presenting a discrete file containing only the eFEDS-XCS clusters was easier.

## Comparison of the eFEDS optically confirmed and XXL-100-GC catalogues

In [ ]:
# sublabel_ypos = -0.115
# sublabel_fsize = 14

# t_frac_bins = np.arange(0, 1.05, 0.03)
# z_bins = np.arange(0, 1.4, 0.1)
# t_bins = np.arange(0, 8, 0.5)

# t_frac_dens = True
# z_dens = True
# t_dens = True

# fig, ax_arr = plt.subplots(ncols=3, figsize=(16, 6))

# for ax in ax_arr:
#     ax.minorticks_on()
#     ax.tick_params(which='both', top=True, right=True, direction='in')

# ax = ax_arr[0]  
# ax.hist(sample['z'], color='firebrick', label='eFEDS', alpha=0.7, bins=z_bins, density=True)
# ax.hist(xxl['z'], color='tab:cyan', label='XXL-100-GC', alpha=0.7, bins=z_bins, density=True)

# ax.set_xlabel(r"z", fontsize=15)
# ax.set_xlim(0)
# ax.set_yticks([])
# ax.set_ylabel("Probability Density", fontsize=15)
# ax.text(0.5, sublabel_ypos, s='a)', horizontalalignment='center', verticalalignment='center', 
#         transform=ax.transAxes, fontsize=sublabel_fsize)
# ax.legend(loc='best', fontsize=14)


# ax = ax_arr[1]
# # ax.minorticks_on()
# # ax.tick_params(which='both', top=True, right=True, direction='in')

# ax.hist(sample['T_300kpc'], color='firebrick', label='eFEDS', alpha=0.7, bins=t_bins, density=True)
# ax.hist(xxl['T300kpc'], color='tab:cyan', label='XXL-100-GC', alpha=0.7, bins=t_bins, density=True)

# ax.set_xlabel(r"T$_{\rm{x, 300kpc}}$ [keV]", fontsize=15)
# ax.set_xlim(0)
# ax.set_yticks([])
# # ax.set_ylabel("Density", fontsize=13)
# ax.legend(loc='best', fontsize=14)
# ax.text(0.5, sublabel_ypos, s='b)', horizontalalignment='center', verticalalignment='center', 
#         transform=ax.transAxes, fontsize=sublabel_fsize)

# ax = ax_arr[2]
# av_frac_err = np.nanmean(sample[['T_300kpc-', 'T_300kpc+']].replace(0, np.NaN).values, axis=1) /sample['T_300kpc']
# print('eFEDS:', np.mean(av_frac_err))
# ax.hist(av_frac_err, bins=t_frac_bins, color='firebrick', alpha=0.7, label='eFEDS', 
#          density=True)

# xxl_frac_err = xxl['e_T300kpc'] / xxl['T300kpc']
# print('XXL:', np.mean(xxl_frac_err))
# ax.hist(xxl_frac_err, bins=t_frac_bins, color='tab:cyan', alpha=0.7, label='XXL-100-GC', density=True)

# # ax.set_xlabel(r"T$_{\rm{x, 300kpc}}$ Fractional Uncertainty", fontsize=15)
# ax.set_xlabel(r"$\Delta \rm{T}_{\rm{x, 300kpc}}$ / $\rm{T}_{\rm{x, 300kpc}}$", fontsize=15)

# ax.set_xlim(0, 0.5)
# ax.set_yticks([])
# ax.text(0.5, sublabel_ypos, s='c)', horizontalalignment='center', verticalalignment='center', 
#         transform=ax.transAxes, fontsize=sublabel_fsize)

# ax.legend(loc='best', fontsize=14)

# plt.tight_layout()
# plt.savefig("output_plots/efeds_xxl_z+t+t_frac.png")
# plt.show()

## Brief exploration of the eFEDS-XMM sample

### Creating a ClusterSample for eFEDS-XMM

### Exposure times of the eFEDS-XMM candidates

## Comparisons of cluster properties measured by eFEDS and XCS

### Creating a ClusterSample for eFEDS-XCS